In [1]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
# from keras import backend as K
# K.set_image_data_format('channels_first')
import cv2
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks import *
from PIL import Image
%matplotlib inline

In [2]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [3]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))    
    return loss

In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

KeyError: 'conv1_w'

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv2.imread('./images/sveta1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

crop_img = img[y:y+h, x:x+w]
res = cv2.resize(crop_img, dsize=(96, 96), interpolation=cv2.INTER_CUBIC)
img = Image.fromarray(res, 'RGB')
img.save('./images/sveta1_cropped.jpg')

In [ ]:
database = {}
database["sveta serikova"] = img_to_encoding('./images/sveta1_cropped.jpg', FRmodel)
# database["borat"] = img_to_encoding("images/sacha_cropped.jpg", FRmodel)

In [ ]:
def verify(image_path, identity, database, model):
    encoding =  img_to_encoding(image_path, model)
    dist = np.linalg.norm(encoding - database[identity])
    if dist < 0.7:
        print("Это правда" + str(identity) + ", проходи!")
        boolean = True
    else:
        print("Это не " + str(identity) + ", оповещаю охрану ")
        boolean = False        
    return dist, boolean

In [ ]:
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    cv2.imshow("Press SPACE to take a picture and ESC to close", frame)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        print("Camera Closed")
        break
    elif k%256 == 32:
        img_name = "webcam_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        res = cv2.imread(img_name)
        img = Image.fromarray(res, 'RGB')
        img.save('./images/webcam.jpg')    
        print("{} has been added".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv2.imread('./images/webcam.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

crop_img = img[y:y+h, x:x+w]
res = cv2.resize(crop_img, dsize=(96, 96), interpolation=cv2.INTER_CUBIC)
img = Image.fromarray(res, 'RGB')
img.save('./images/webcam.jpg')

verify("./images/webcam.jpg", 'sveta serikova' , database, FRmodel)

Если отказывает мне же, то увличить пороговое значение до 1

In [ ]:
img1 = cv2.imread('images/webcam.jpg')
img2 = cv2.imread('images/san_cropped.jpg')
print("############## IT'S A MATCH ###################")
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img1)
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img2)